## Efficient frontier

#### Portfolio analysis

In [1]:
import pandas as pd
import numpy as np
import edhec_risk_kit as erk



In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

This time, we will use another file. Since we are going to use it more than once, I created a function that returns the formatted DataFrame.

In [ ]:
ind = erk.get_ind_returns()

In [ ]:
ind.head()

Let's compute the drawdowns from the food column:

In [ ]:
erk.drawdown(ind["Food"])["Drawdown"].plot(figsize=(12,6))

Let's now have a look at the VaR for some indices.

In [ ]:
erk.var_comparison(ind[["Food","Smoke","Coal","Beer","Fin"]],plot=True)

As you can see, it seems that Cornish-Fisher VaR gives much lower VaR, especially with the coal. In general, we can see the the lowest VaR is for the food. Another thing that we should point out is that Food and Smoke have the most seemingly normal behaviour (Gaussian and Cornish-Fisher VaR are very similar). We can check if this is true by having a look at the skewness and kurtosis. In some sense, this can make sense, as both smoking and eating  should not have frequent extreme deviations (steady amount of consumers and consumption). However, I would guess that if we divide food into different industries, we would see a lot of deviations from normality.

In [ ]:
erk.kurtosis(ind[["Food","Smoke","Coal","Beer","Fin"]])

In [ ]:
erk.skewness(ind[["Food","Smoke","Coal","Beer","Fin"]]) 

In [ ]:
ind[["Food","Smoke","Coal","Beer","Fin"]].std(ddof=0)

Let's have a look at the Cornish_Fisher VaR for all industries:

In [ ]:
erk.var_cornish_fisher(ind).sort_values().plot.bar()


We can compute some statistics for the returns, for example, the Sharpe ratio. We will assume a risk-free rate of 0.03. Since the periods of the given returns are months, we have 12 periods per year:

In [ ]:
erk.sharpe_ratio(ind,0.03,12).sort_values().plot.bar(title="Industry Sharpe Ratios 1926-2018",color="green")

So food, smoke and health industry have given really good results, compared to some industries such as coal or steel. Beer, which is one of the Top 3 in the welath index has also given good Sharpe ratios. Let's have a look at Sharpe ratio from 2000-2018, which might be more representative.

In [ ]:
erk.sharpe_ratio(ind["2000":],0.03,12).sort_values().plot.bar(title="Industry Sharpe Ratios 2000-2018",color="green")

It is interesting to see how Meals industry has tremendously improved its SR, but also the smoke industry. Another interesting fact is that both food and meal industry have more or less the same SR. What we can do next is calculate the difference between 2000-2018 and 1926-2018. This allows us to see how the industries have developed with time.

In [ ]:
(erk.sharpe_ratio(ind["2000":],0.03,12)-erk.sharpe_ratio(ind,0.03,12)).sort_values().plot.bar(title=r"(Industry SR$_{2000-2018}$ / Industry SR$_{1926-2018})-1$",color="green")

As we said before, Meals and Smoke industries have improved a lot. Food and Beer are pretty steady in time, while Telecommunications, Business Equities, Automovilistics and Books have taken an extremely big hit. To be honest, none of them are surprising.

#### Efficient frontier

We will study the returns from 1995 to 2000. As we have seen in classes, our task now is to find the sweet spot that 

In [ ]:
er = erk.annualized_rets(ind["1995":"2000"],12)

Now we must calculate the covariance matrix, which can be calculated with the built-in function cov():

In [ ]:
cov = ind["1995":"2000"].cov()

In [ ]:
cov